In [7]:
import pandas as pd
import numpy as np
import torch as th
import zipfile
# from tqdm.auto import tqdm, trange # original
from tqdm.notebook import tqdm, trange # fix for newline issue?
from glob import glob
from joblib import delayed, Parallel
import matplotlib.pyplot as plt
import torch_geometric as pyg
import zipfile, os
import torch
import copy
from torch import nn
import networkx as nx
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import trange
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d
from sklearn.neighbors import KNeighborsRegressor,RadiusNeighborsRegressor

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print("Using device:", device)

import torch
import torch_geometric
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import MessagePassing
from torch.nn import Sequential as Seq, Linear, ReLU, Sigmoid
import torch.optim as optim
import joblib
import gc
from scipy.optimize import root_scalar

pd.set_option("display.max_columns", 100)
PATH_DATA0 = '../data/00.01'
PATH_DATA = '../data/00.02'
RANDOM_SEED =0
np.random.seed(RANDOM_SEED)  
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Using device: mps


# Hyperparameters

In [8]:
CRITERION = nn.BCEWithLogitsLoss()
LR = 0.001
TOLERANCE = 20
LR_TOLERANCE= 5
MAX_EPOCHS = 200
BATCH_SIZE =2

# Loaders

In [9]:
loader_train = pyg.loader.DataLoader(
    pd.read_pickle(os.path.join(PATH_DATA0, 'graphs','max_prob_10_subsample_0.1','graphs_train.pkl')).tolist(),
    batch_size = BATCH_SIZE,shuffle = True)
loader_val = pyg.loader.DataLoader(
    pd.read_pickle(os.path.join(PATH_DATA0, 'graphs','max_prob_10_subsample_0.1','graphs_val.pkl')).tolist(),batch_size = BATCH_SIZE
    ,shuffle = False)
loader_test = pyg.loader.DataLoader(
    pd.read_pickle(os.path.join(PATH_DATA0, 'graphs','max_prob_10_subsample_0.1','graphs_test.pkl')).tolist(),batch_size = BATCH_SIZE
    ,shuffle = False)

# Model Architecture

In [10]:
class RelationalModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers):
        super(RelationalModel, self).__init__()

        layers = [nn.Linear(input_size, hidden_size), 
                 nn.ReLU()]
        if n_layers>=2:
            for _ in range(n_layers - 1):
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_size, output_size))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)
class ObjectModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, n_layers):
        super(ObjectModel, self).__init__()

        layers = [nn.Linear(input_size, hidden_size), 
                 nn.ReLU()]
        if n_layers>=2:
            for _ in range(n_layers - 1):
                layers.append(nn.Linear(hidden_size, hidden_size))
                layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_size, output_size))

        self.layers = nn.Sequential(*layers)

    def forward(self, C):
        return self.layers(C)
class InteractionNetwork(MessagePassing):
    def __init__(self, hidden_size, n_layers):
        super(InteractionNetwork, self).__init__(aggr='add', 
                                                 flow='source_to_target')
        self.R1 = RelationalModel(10, 4, hidden_size, n_layers)
        self.O = ObjectModel(7, 3, hidden_size, n_layers)
        self.R2 = RelationalModel(10, 1, hidden_size, n_layers)
        self.E: Tensor = Tensor()

    def forward(self, x: Tensor, edge_index: Tensor, edge_attr: Tensor) -> Tensor:

        # propagate_type: (x: Tensor, edge_attr: Tensor)
        x_tilde = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)

        m2 = torch.cat([x_tilde[edge_index[1]],
                        x_tilde[edge_index[0]],
                        self.E], dim=1)
        return self.R2(m2)

    def message(self, x_i, x_j, edge_attr):
        # x_i --> incoming
        # x_j --> outgoing        
        m1 = torch.cat([x_i, x_j, edge_attr], dim=1)
        self.E = self.R1(m1)
        return self.E

    def update(self, aggr_out, x):
        c = torch.cat([x, aggr_out], dim=1)
        return self.O(c) 

# Functions

In [11]:
def count_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return trainable_params
def evaluate(model, loader):
    model.eval()
    preds, actuals = [],[]
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            preds.append(torch.sigmoid(model(batch.x, batch.edge_index, batch.edge_attr)))
            actuals.append(batch.y)
        preds = torch.cat(preds)
        actuals = torch.cat(actuals)
        acc = ((preds>0.5)==(actuals>0.5)).type(torch.float).mean().item()
        entropy = CRITERION(preds, actuals.float()).item()
    model.train()
    return preds.cpu().numpy(), actuals.cpu().numpy(), acc, entropy
def train_epoch(model, loader_train,optimizer):
    model.train()
    train_loss = 0.0
    for batch in tqdm(loader_train, leave = False):
        batch = batch.to(device)
        optimizer.zero_grad()
        output = model(batch.x, batch.edge_index, batch.edge_attr)
        loss = CRITERION(output, batch.y.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch.num_graphs
    train_loss /= len(loader_train)    
    return train_loss


# Train

In [ ]:
def f(n_layers,target_params):
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print(n_layers, target_params)
    # Find out the hyperparameteres yielding #params = target_params
    def objective(h):
        return count_parameters(InteractionNetwork(int(h),n_layers)) - target_params
    optimal_h = int(root_scalar(objective, bracket=[1, 50000], method='bisect').root)
    print(f"(1) Optimal h: {optimal_h}")
    optimal_h= pd.Series({optimal_h:target_params-count_parameters(InteractionNetwork(optimal_h, n_layers)),
                optimal_h-1:target_params-count_parameters(InteractionNetwork(optimal_h-1,n_layers)),
                optimal_h+1:target_params-count_parameters(InteractionNetwork(optimal_h+1,n_layers))}).abs().idxmin()
    print(f"(2) Optimal h: {optimal_h}")
    
    model = InteractionNetwork(optimal_h,n_layers).to(device)
    lr = LR
    optimizer = optim.Adam(model.parameters(), lr=LR)
    best_val_loss = float('inf')
    epochs_no_improve, epochs_no_improve2 = 0,0
    best_model_state = None
    stats = []
    best = None
    # Print header once
    print(f"{'Epoch':>5} | {'Train Loss':>10} | {'Val Loss':>9} | {'Val Acc':>8} | {'Test Acc':>9}")
    print("-" * 50)
    for epoch in trange(MAX_EPOCHS):
        train_loss = train_epoch(model, loader_train, optimizer)   
        preds_val, actuals_val, acc_val, val_loss = evaluate(model,loader_val)
        preds_test, actuals_test, acc_test, test_loss = evaluate(model,loader_test)
        
        stats.append({'train_loss':train_loss, 'val_loss':val_loss, 'acc_val':acc_val, 'acc_test':acc_test})
        if val_loss < best_val_loss: 
            print(f"{epoch+1:5d} | {train_loss:10.4f} | {val_loss:9.4f} | {acc_val:8.4f} | {acc_test:9.4f} *")
            best_val_loss = val_loss
            epochs_no_improve = 0
            epochs_no_improve2 = 0
            best = {'model_state': {k: v.cpu() for k, v in model.state_dict().items()},
                    'preds_test':preds_test, 'preds_val':preds_val}        
        else:
            print(f"{epoch+1:5d} | {train_loss:10.4f} | {val_loss:9.4f} | {acc_val:8.4f} | {acc_test:9.4f}")
            epochs_no_improve += 1
            epochs_no_improve2 += 1

        if epochs_no_improve >= TOLERANCE:
            print(f"Early stopping at epoch {epoch+1}")
            break
        if epochs_no_improve2 >= LR_TOLERANCE:
            if lr >=1.0e-8:
                lr/=10
            print(f"LR reduction to {lr}")
    best['stats'] = stats
    os.makedirs(PATH_DATA, exist_ok=True)
    joblib.dump(best, os.path.join(PATH_DATA, f"{n_layers}_{target_params}.pkl"))
    
    stats = pd.DataFrame(stats)
    stats[['train_loss','val_loss']].plot(figsize = (15,4))
    plt.show()
    stats[['acc_val','acc_test']].plot(figsize = (15,4))
    plt.show()
    del model
    del train_loss
    del optimizer

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
Parallel(n_jobs=1)(delayed(f)(n_layers,target_params)
                    for n_layers in tqdm([2,3,4])
                    for target_params in tqdm([1_000_000,100_000,
                                               500_000,]))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

2 1000000
(1) Optimal h: 571
(2) Optimal h: 571
Epoch | Train Loss |  Val Loss |  Val Acc |  Test Acc
--------------------------------------------------


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/283 [00:00<?, ?it/s]

...

    7 |     0.2472 |    0.4008 |   0.9531 |    0.9541


  0%|          | 0/283 [00:00<?, ?it/s]

    8 |     0.2222 |    0.3999 |   0.9556 |    0.9569 *


  0%|          | 0/283 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Summary of the Results

In [23]:
val = pd.read_pickle(os.path.join(PATH_DATA0, 'graphs','max_prob_10_subsample_0.1','graphs_val.pkl'))
test = pd.read_pickle(os.path.join(PATH_DATA0, 'graphs','max_prob_10_subsample_0.1','graphs_test.pkl'))
val_y = torch.cat([i.y for i in val.values]).numpy()
test_y = torch.cat([i.y for i in test.values]).numpy()

In [ ]:
# Run inference on validation and test sets
def run_inference(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            preds.append(torch.sigmoid(model(batch.x, batch.edge_index, batch.edge_attr)))
        preds = torch.cat(preds)
    return preds.cpu().numpy()

val = run_inference(model, loader_val)
test = run_inference(model, loader_test)

(141, 1)

In [ ]:
results = dict()
for p in glob(os.path.join(PATH_DATA,'*.pkl')): 
    preds_test = joblib.load(p)['preds_test']>=0.5
    preds_val =joblib.load(p)['preds_val']>=0.5
    print(preds_test)
    results[p]={'test_accuracy': (preds_test==test_y).astype(float).mean(),
     'val_accuracy':(preds_val==val_y).astype(float).mean()}
    results = pd.DataFrame(results)
results.columns = [os.path.split(i)[-1].replace('.pkl','') for i in results.columns]
results = results.T.sort_values('val_accuracy', ascending = False)
results.index = pd.MultiIndex.from_tuples([tuple(i.split('_')) for i in results.index])
results.index.names = ['#layers','#params']
results.to_csv(os.path.join(PATH_DATA, 'results.csv'))

(177, 1)


ValueError: operands could not be broadcast together with shapes (177,1) (5472684,1) 

In [51]:
results

test_accuracy  val_accuracy
#layers #params                             
4       1000000       0.994750      0.994493
        500000        0.994401      0.994209
3       1000000       0.993834      0.993719
4       100000        0.993913      0.993667
3       500000        0.993656      0.993433
2       1000000       0.992984      0.992853
        500000        0.992865      0.992497
3       100000        0.992658      0.992375
2       100000        0.990710      0.990459

In [52]:
print('The Accuracy', results.iloc[0]['test_accuracy'])

The Accuracy 0.9947501079908871


-----------------------------------

In [7]:
asdfasdfasfd

NameError: name 'asdfasdfasfd' is not defined

In [ ]:


for n_layers in tqdm([3,2,4]): 
    for target_params in tqdm([100_000,500_000,1_000_000]): 
        print(n_layers, target_params)
        # Find out the hyperparameteres yielding #params = target_params
        def objective(h):
            return count_parameters(InteractionNetwork(int(h),n_layers)) - target_params
        optimal_h = int(root_scalar(objective, bracket=[1, 3000], method='bisect').root)
        optimal_h= pd.Series({optimal_h:target_params-count_parameters(InteractionNetwork(optimal_h, n_layers)),
                    optimal_h-1:target_params-count_parameters(InteractionNetwork(optimal_h-1,n_layers)),
                    optimal_h+1:target_params-count_parameters(InteractionNetwork(optimal_h+1,n_layers))}).abs().idxmin()
        
        model = InteractionNetwork(optimal_h,n_layers).to(device)
        lr = LR
        optimizer = optim.Adam(model.parameters(), lr=LR)
        best_val_loss = float('inf')
        epochs_no_improve, epochs_no_improve2 = 0,0
        best_model_state = None
        stats = []
        best = None
        # Print header once
        print(f"{'Epoch':>5} | {'Train Loss':>10} | {'Val Loss':>9} | {'Val Acc':>8} | {'Test Acc':>9}")
        print("-" * 50)
        for epoch in trange(MAX_EPOCHS):
            train_loss = train_epoch(model, loader_train)   
            preds_val, actuals_val, acc_val, val_loss = evaluate(model,loader_val)
            preds_test, actuals_test, acc_test, test_loss = evaluate(model,loader_test)
            
            stats.append({'train_loss':train_loss, 'val_loss':val_loss, 'acc_val':acc_val, 'acc_test':acc_test})
            if val_loss < best_val_loss: 
                print(f"{epoch+1:5d} | {train_loss:10.4f} | {val_loss:9.4f} | {acc_val:8.4f} | {acc_test:9.4f} *")
                best_val_loss = val_loss
                epochs_no_improve = 0
                epochs_no_improve2 = 0
                best = {'model_state': {k: v.cpu() for k, v in model.state_dict().items()},
                        'preds_test':preds_test, 'preds_val':preds_val}        
            else:
                print(f"{epoch+1:5d} | {train_loss:10.4f} | {val_loss:9.4f} | {acc_val:8.4f} | {acc_test:9.4f}")
                epochs_no_improve += 1
                epochs_no_improve2 += 1
        
            if epochs_no_improve >= TOLERANCE:
                print(f"Early stopping at epoch {epoch+1}")
                break
            if epochs_no_improve2 >= LR_TOLERANCE:
                if lr >=1.0e-8:
                    lr/=10
                print(f"LR reduction to {lr}")
        os.makedirs(PATH_DATA, exist_ok=True)
        joblib.dump(best, os.path.join(PATH_DATA, f"{n_layers}_{target_params}.pkl"))
        
        stats = pd.DataFrame(stats)
        stats[['train_loss','val_loss']].plot(figsize = (15,4))
        plt.show()
        stats[['acc_val','acc_test']].plot(figsize = (15,4))
        plt.show()